In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
data = pd.read_csv('Dataset - NLP Assignment.csv')
data.head()

,ID,MEMBER_ID,REASONNPSSCORE__C
0,a2p1U000000RowfQAC,0011U00000rjFKdQAM,"I showed up for my appointment, but they had m..."
1,a2p1U000000RqQqQAK,0011U00000riCSHQA2,"Staff was polite, courteous, and on time"
2,a2p1U000000RqXyQAK,0011U00000riTw7QAE,Overall care is great! It's wonderful to be a...
3,a2p1U000000Rq1LQAS,0011U00000rhu8eQAA,Like the doctor and staff at this location. Ea...
4,a2p1U000000RpiuQAC,0011U00000rk4SHQAY,The convenience and the doctors


In [11]:
data.drop(columns=['ID','MEMBER_ID'],inplace=True)

In [12]:
data

,REASONNPSSCORE__C
0,"I showed up for my appointment, but they had m..."
1,"Staff was polite, courteous, and on time"
2,Overall care is great! It's wonderful to be a...
3,Like the doctor and staff at this location. Ea...
4,The convenience and the doctors
...,...
3807,yes
3808,Very skeptical that you will soon be without a...
3809,"Doctor Malfese is easily accessible, staff is ..."
3810,Friendly Staff that can be reached at any time


In [13]:
data.dtypes

REASONNPSSCORE__C    object
dtype: object

In [14]:
data['REASONNPSSCORE__C'] = data['REASONNPSSCORE__C'].astype(str)

In [15]:
data.dtypes

REASONNPSSCORE__C    object
dtype: object

In [16]:
# Import the necessary modules for LDA with gensim
from gensim import matutils, models
import scipy.sparse

In [17]:
# Create a new document-term matrix 
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# create a document-term matrix 
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data['REASONNPSSCORE__C'])
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data.index
data_dtmn

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,01,06,0800,10,100,102,1056am,10pm,11,...,years,yep,yes,yesterday,young,younger,yte,zero,zone,zoster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3807,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3808,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3809,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3810,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [19]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.024*"doctor" + 0.016*"appointment" + 0.014*"dr" + 0.010*"staff" + 0.010*"did" + 0.010*"nice" + 0.008*"office" + 0.007*"didn" + 0.007*"told" + 0.007*"day"'),
 (1,
  '0.026*"service" + 0.024*"good" + 0.022*"friendly" + 0.019*"doctor" + 0.018*"dr" + 0.018*"staff" + 0.018*"great" + 0.017*"care" + 0.016*"convenient" + 0.014*"easy"')]

In [20]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.032*"doctor" + 0.024*"good" + 0.023*"dr" + 0.020*"great" + 0.016*"staff" + 0.016*"service" + 0.015*"appointment" + 0.013*"easy" + 0.013*"visit" + 0.012*"care"'),
 (1,
  '0.033*"friendly" + 0.032*"convenient" + 0.029*"staff" + 0.023*"service" + 0.020*"dr" + 0.016*"quick" + 0.015*"care" + 0.012*"easy" + 0.012*"doctors" + 0.012*"professional"'),
 (2,
  '0.022*"doctor" + 0.018*"appointment" + 0.010*"told" + 0.010*"day" + 0.010*"xyz" + 0.008*"office" + 0.008*"did" + 0.007*"called" + 0.007*"didn" + 0.007*"clinic"')]

In [21]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.038*"easy" + 0.028*"convenient" + 0.027*"appointment" + 0.026*"good" + 0.024*"doctor" + 0.021*"service" + 0.016*"appointments" + 0.015*"dr" + 0.015*"care" + 0.013*"convenience"'),
 (1,
  '0.018*"xyz" + 0.015*"doctor" + 0.012*"office" + 0.011*"told" + 0.011*"experience" + 0.011*"blood" + 0.010*"visit" + 0.009*"appointment" + 0.009*"called" + 0.008*"did"'),
 (2,
  '0.046*"dr" + 0.033*"doctor" + 0.023*"care" + 0.020*"feel" + 0.017*"really" + 0.013*"great" + 0.009*"personal" + 0.008*"visit" + 0.008*"patient" + 0.008*"xyz"'),
 (3,
  '0.061*"staff" + 0.059*"friendly" + 0.026*"service" + 0.025*"nice" + 0.019*"quick" + 0.018*"great" + 0.018*"doctor" + 0.017*"professional" + 0.015*"helpful" + 0.013*"thorough"')]

## Topic Modeling - Attempt #2 (Nouns Only)

In [22]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [23]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data['REASONNPSSCORE__C'].apply(nouns))
data_nouns

,REASONNPSSCORE__C
0,appointment doctor visit doctor staff office e...
1,Staff time
2,care minutes Dr
3,doctor staff location Easy appointment
4,convenience doctors
...,...
3807,yes
3808,Dr. nobody phone
3809,Doctor Malfese staff locations
3810,Staff time


In [24]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns['REASONNPSSCORE__C'])
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,19,2146826259,30,3days,45,_n,a1c,abcesses,abilities,ability,...,xyz,xyzs,year,years,yes,yesterday,yte,zero,zone,zoster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3807,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3808,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3809,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3810,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [26]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.054*"staff" + 0.046*"dr" + 0.045*"service" + 0.026*"care" + 0.025*"office" + 0.025*"doctors" + 0.017*"xyz" + 0.016*"experience" + 0.014*"blood" + 0.012*"convenience"'),
 (1,
  '0.077*"doctor" + 0.041*"appointment" + 0.018*"convenient" + 0.018*"dr" + 0.017*"visit" + 0.016*"health" + 0.015*"xyz" + 0.014*"appointments" + 0.013*"day" + 0.012*"care"')]

In [27]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.090*"service" + 0.029*"doctor" + 0.027*"experience" + 0.018*"questions" + 0.016*"great" + 0.016*"appointment" + 0.015*"easy" + 0.014*"attention" + 0.014*"quick" + 0.012*"wait"'),
 (1,
  '0.064*"dr" + 0.061*"staff" + 0.040*"appointment" + 0.027*"visit" + 0.027*"doctor" + 0.023*"xyz" + 0.022*"office" + 0.019*"appointments" + 0.015*"day" + 0.014*"care"'),
 (2,
  '0.074*"doctor" + 0.037*"care" + 0.031*"convenient" + 0.027*"doctors" + 0.020*"health" + 0.020*"location" + 0.018*"xyz" + 0.013*"access" + 0.013*"office" + 0.013*"nurse"')]

In [28]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.103*"doctor" + 0.049*"appointment" + 0.028*"experience" + 0.026*"xyz" + 0.025*"care" + 0.021*"convenience" + 0.018*"office" + 0.016*"patient" + 0.014*"quick" + 0.014*"phone"'),
 (1,
  '0.117*"staff" + 0.033*"visit" + 0.025*"appointments" + 0.025*"service" + 0.023*"office" + 0.018*"access" + 0.018*"attention" + 0.018*"easy" + 0.016*"blood" + 0.015*"ease"'),
 (2,
  '0.120*"dr" + 0.071*"service" + 0.038*"care" + 0.015*"great" + 0.012*"xyz" + 0.012*"appt" + 0.010*"appointment" + 0.009*"day" + 0.008*"customer" + 0.008*"visit"'),
 (3,
  '0.053*"doctor" + 0.040*"doctors" + 0.035*"convenient" + 0.029*"health" + 0.024*"appointment" + 0.021*"work" + 0.021*"office" + 0.020*"xyz" + 0.020*"location" + 0.013*"wait"')]

# Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [29]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [30]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data['REASONNPSSCORE__C'].apply(nouns_adj))
data_nouns_adj

,REASONNPSSCORE__C
0,appointment double doctor expedited visit doct...
1,Staff polite courteous time
2,Overall care great wonderful able 45-60 minute...
3,doctor staff location Easy appointment
4,convenience doctors
...,...
3807,yes
3808,skeptical Dr. long nobody phone
3809,Doctor Malfese accessible staff friendly helpf...
3810,Friendly Staff time


In [31]:
# Recreate a document-term matrix with nouns and adjative
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns_adj.REASONNPSSCORE__C)
data_dtmna = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,06,10,15,15th,19,1st,20,2021,21,2146826259,...,yearly,years,yes,yesterday,young,younger,yte,zero,zone,zoster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3807,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3808,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3809,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3810,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvn.vocabulary_.items())

In [33]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.032*"appointment" + 0.031*"doctor" + 0.029*"dr" + 0.015*"xyz" + 0.013*"visit" + 0.011*"day" + 0.011*"office" + 0.009*"able" + 0.008*"phone" + 0.007*"clinic"'),
 (1,
  '0.038*"staff" + 0.033*"service" + 0.032*"good" + 0.032*"doctor" + 0.024*"friendly" + 0.023*"great" + 0.022*"easy" + 0.022*"convenient" + 0.020*"care" + 0.017*"dr"')]

In [34]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.040*"good" + 0.039*"great" + 0.032*"appointment" + 0.031*"service" + 0.026*"dr" + 0.019*"xyz" + 0.017*"doctor" + 0.016*"experience" + 0.014*"doctors" + 0.012*"care"'),
 (1,
  '0.045*"doctor" + 0.022*"visit" + 0.020*"care" + 0.016*"health" + 0.015*"xyz" + 0.015*"dr" + 0.013*"office" + 0.009*"patient" + 0.008*"appointment" + 0.007*"phone"'),
 (2,
  '0.060*"staff" + 0.038*"friendly" + 0.036*"convenient" + 0.034*"easy" + 0.029*"doctor" + 0.029*"dr" + 0.025*"service" + 0.024*"nice" + 0.018*"quick" + 0.018*"office"')]

In [35]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.036*"nice" + 0.028*"staff" + 0.021*"friendly" + 0.018*"doctors" + 0.017*"knowledgeable" + 0.016*"helpful" + 0.016*"doctor" + 0.014*"dr" + 0.014*"wait" + 0.013*"clinic"'),
 (1,
  '0.072*"doctor" + 0.023*"appointment" + 0.017*"staff" + 0.016*"office" + 0.015*"blood" + 0.014*"nurse" + 0.014*"dr" + 0.013*"phone" + 0.011*"xyz" + 0.009*"visit"'),
 (2,
  '0.052*"good" + 0.040*"convenient" + 0.035*"service" + 0.034*"great" + 0.029*"dr" + 0.026*"doctor" + 0.024*"xyz" + 0.017*"experience" + 0.015*"location" + 0.014*"staff"'),
 (3,
  '0.050*"easy" + 0.035*"appointment" + 0.034*"dr" + 0.031*"service" + 0.025*"quick" + 0.024*"care" + 0.020*"staff" + 0.018*"convenience" + 0.017*"office" + 0.014*"friendly"')]

## Identify Topics in Each Document

the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [36]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=100)
ldana.print_topics()

[(0,
  '0.076*"dr" + 0.029*"appointment" + 0.020*"xyz" + 0.018*"care" + 0.015*"visit" + 0.015*"day" + 0.012*"able" + 0.011*"health" + 0.010*"phone" + 0.010*"clinic"'),
 (1,
  '0.054*"doctor" + 0.053*"convenient" + 0.033*"office" + 0.023*"appointment" + 0.022*"easy" + 0.019*"visit" + 0.019*"location" + 0.015*"access" + 0.013*"personal" + 0.012*"attention"'),
 (2,
  '0.082*"staff" + 0.072*"service" + 0.069*"good" + 0.052*"friendly" + 0.037*"great" + 0.033*"nice" + 0.029*"easy" + 0.022*"professional" + 0.018*"quick" + 0.018*"helpful"'),
 (3,
  '0.080*"doctor" + 0.023*"xyz" + 0.012*"work" + 0.012*"new" + 0.012*"health" + 0.011*"patient" + 0.011*"nurse" + 0.010*"doctors" + 0.010*"room" + 0.010*"care"')]

In [37]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 4
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data['REASONNPSSCORE__C'])
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
 
text = "The economy is working better than ever"
x = lda_model.transform(vectorizer.transform([text]))[0]
print(x, x.sum())

[0.03333767 0.03333333 0.36665258 0.36666665 0.03333641 0.03333333
 0.03333333 0.03333333 0.03334001 0.03333333] 0.9999999999999998


In [38]:
import warnings
warnings.filterwarnings("ignore")

In [39]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2      32.420952   8.115230       1        1  16.527277
8       0.648955  71.707100       2        1  13.129131
3      71.019684   9.971006       3        1  11.480450
4       3.785981  33.185135       4        1  11.350258
9     -42.666500  -9.222070       5        1   9.622394
0      -3.621689 -43.526516       6        1   9.063077
5      -3.632402  -4.171708       7        1   8.705056
6     -36.584400  35.804413       8        1   8.362748
7      44.391380  50.615383       9        1   7.400463
1      40.235809 -31.591665      10        1   4.359147, topic_info=           Term        Freq       Total Category  logprob  loglift
325    friendly  410.000000  410.000000  Default  30.0000  30.0000
731     service  397.000000  397.000000  Default  29.0000  29.0000
766       staff  445.000000  445.000000  Default  28.0000  28.0000
346        good  383.000000  383.000000  Default  27.0000  27.0000
172  convenient  280.000000  280.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
418         isn   13.405982   19.719872  Topic10  -4.4878   2.7470
915         xyz   65.521564  328.930642  Topic10  -2.9012   1.5194
230         don   31.071231  207.081755  Topic10  -3.6473   1.2361
663      reason   13.802655   30.449850  Topic10  -4.4587   2.3417
468        long   13.368265   94.328394  Topic10  -4.4907   1.1790

[529 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         5  0.946282     ability
1         1  0.665264        able
1         4  0.328197        able
2         2  0.993770      access
4         7  0.925746  accessible
...     ...       ...         ...
915       7  0.130727         xyz
915       9  0.100325         xyz
915      10  0.200650         xyz
917       1  0.989947       years
919       4  0.915510   yesterday

[653 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 4, 5, 10, 1, 6, 7, 8, 2])

These four topics look pretty decent. Let's settle on these for now.
* Topic 0: appointment,doctor
* Topic 1: Time,appointment
* Topic 2: xyz,doctor
* Topic 3: profanity
* Topic 4: staff,friendly,good,service
* Topic 5: feel,like
* Topic 6: time,doctor
* Topic 7: Know,don
* Topic 8: care,health
* topic 9: doctor,like
* topic10: convenient,nice,quick
## seems like Almost all patient are convenient with the staff and doctor